# **Cardiac Data Pre-Processing**

This notebook outlines pre-processing procedures on electrocardiograph (ECG) and photoplethysmograph (PPG) data.

## Imports

In [1]:
import pandas as pd
from utils import interactive_plot
from ipywidgets import interact
from heartview import *

## ECG Data
Load the sample ECG data.

In [2]:
ecg = pd.read_csv('data/sample_ecg.csv')
ecg.head()

,Segment,Timestamp,ECG
0,1,2017-03-18 14:57:18.000000,0.110697
1,1,2017-03-18 14:57:18.000977,0.110697
2,1,2017-03-18 14:57:18.001953,0.102508
3,1,2017-03-18 14:57:18.002930,0.078527
4,1,2017-03-18 14:57:18.003906,0.081159


Get the ECG sampling frequency.

In [3]:
ecg.Timestamp = pd.to_datetime(ecg.Timestamp)
ecg_fs = round(1 / (ecg.Timestamp.iloc[1] - ecg.Timestamp.iloc[0]).total_seconds())
ecg_fs

1024

### ECG Filtering

Apply a 2nd-order elliptic bandpass filter with cutoff frequencies of 1 Hz to 15 Hz to the signal.

In [4]:
ecg['Filtered'] = filter_ecg(signal = ecg['ECG'], fs = ecg_fs)

View the filtered signal.

In [5]:
interactive_plot(ecg, 'Timestamp', ['ECG', 'Filtered']);

interactive(children=(IntSlider(value=1, description='Segment:', max=15, min=1), Output()), _dom_classes=('wid…

### ECG Beat Detection

In [6]:
detect_beats = ECGBeatDetectors(fs = ecg_fs, preprocessed = True)

1. Manikandan & Soman (2012) algorithm  
**Reference:** Manikandan, R., & Soman, K. P. (2012). A novel method for detecting R-peaks in electrocardiogram (ECG) signal. _Biomedical Signal Processing and Control, 7_(2), 118-128.

In [7]:
ecg_beats = detect_beats.manikandan(ecg['Filtered'])
ecg.loc[ecg_beats, 'Beat_Manikandan'] = 1
interactive_plot(ecg, 'Timestamp', 'Filtered', beats = 'Beat_Manikandan');

interactive(children=(IntSlider(value=1, description='Segment:', max=15, min=1), Output()), _dom_classes=('wid…

2. Engelse & Zeelenberg (1979) algorithm  
**Reference:** Engelse, W. A. & Zeelenberg, C. (1979). A single scan algorithm for QRS-detection and feature extraction. _Computers in Cardiology_, 6, 37-42.

In [8]:
ecg_beats = detect_beats.engzee(ecg['Filtered'])
ecg.loc[ecg_beats, 'Beat_Engzee'] = 1
interactive_plot(ecg, 'Timestamp', 'Filtered', beats = 'Beat_Engzee');

interactive(children=(IntSlider(value=1, description='Segment:', max=15, min=1), Output()), _dom_classes=('wid…

3. Pan & Tompkins (1985) algorithm  
**Reference:** Pan, S. J., & Tompkins, W. J. (1985). A real-time QRS detection algorithm. _IEEE Transactions on Biomedical Engineering, 32_(3), 230-236.

In [9]:
ecg_beats = detect_beats.pantompkins(ecg['Filtered'])
ecg.loc[ecg_beats, 'Beat_PanTompkins'] = 1
interactive_plot(ecg, 'Timestamp', 'Filtered', beats = 'Beat_PanTompkins');

interactive(children=(IntSlider(value=1, description='Segment:', max=15, min=1), Output()), _dom_classes=('wid…

## PPG Data
Load the sample PPG data.

In [10]:
ppg = pd.read_csv('data/sample_ppg.csv')
ppg.head()

,Segment,Timestamp,BVP
0,1,2021-11-15 22:59:26.000000,99.67
1,1,2021-11-15 22:59:26.015625,99.74
2,1,2021-11-15 22:59:26.031250,99.43
3,1,2021-11-15 22:59:26.046875,97.86
4,1,2021-11-15 22:59:26.062500,93.65


Get the PPG sampling frequency.

In [11]:
ppg.Timestamp = pd.to_datetime(ppg.Timestamp)
ppg_fs = round(1 / (ppg.Timestamp.iloc[1] - ppg.Timestamp.iloc[0]).total_seconds())
ppg_fs

64

### PPG Filtering

Apply a 2nd-order Chebyshev Type II filter and moving average filter with a window size of 0.25 second to the signal.

In [12]:
ppg['Filtered'] = filter_ppg(ppg['BVP'], fs = ppg_fs, order = 2, window_len = 0.25)
interactive_plot(ppg, 'Timestamp', ['BVP', 'Filtered'], color = ['pink', 'tomato']);

interactive(children=(IntSlider(value=1, description='Segment:', max=20, min=1), Output()), _dom_classes=('wid…

### PPG Beat Detection

In [13]:
detect_beats = PPGBeatDetectors(fs = ppg_fs, preprocessed = True)

1. Adaptive thresholding algorithm

In [14]:
ppg_beats = detect_beats.adaptive_threshold(ppg['Filtered'])
ppg.loc[ppg_beats, 'Beat_AdaptThresh'] = 1
interactive_plot(ppg, 'Timestamp', 'Filtered', beats = 'Beat_AdaptThresh', color = 'tomato');

interactive(children=(IntSlider(value=1, description='Segment:', max=20, min=1), Output()), _dom_classes=('wid…

2. Event-related moving average (ERMA) algorithm  
**Reference:** Elgendi, M., et al. (2013). Systolic peak detection in acceleration photoplethysmograms measured from emergency responders in tropical conditions. _PLoS ONE, 8_(10), e76585.

In [15]:
ppg_beats = detect_beats.erma(ppg['Filtered'])
ppg.loc[ppg_beats, 'Beat_ERMA'] = 1
interactive_plot(ppg, 'Timestamp', 'Filtered', beats = 'Beat_ERMA', color = 'tomato');

interactive(children=(IntSlider(value=1, description='Segment:', max=20, min=1), Output()), _dom_classes=('wid…

## Save Data

In [16]:
ecg.to_csv('data/sample_ecg_preprocessed.csv', index = False)
ppg.to_csv('data/sample_ppg_preprocessed.csv', index = False)